In [ ]:
'''
Sia f(x) = 8cos^2(x) exp[-(x-1)^2/ 2] + 5/2 sin^2(3/2 x) exp[-(x+3)^2/ 8]
definita in [a,b] con a = -12 , b = 7.

-si rappresenti il grafico di f;
-si definisca la derivata f' e se ne rappresenti il grafico;
-si determinino le coordinate del punto di massimo assoluto di f;
-si determinino gli zeri di f' in [a,b];
-si determini l'integrale di f definito in [a,b] con i tre metodi (trapezi, Monte Carlo, Media);
-si definisca la primitiva di f, F(x), rispetto x0 = -12. Se ne rappresenti il grafico;
-si produca un file di testo con le colonne x, f(x), f'(x), F(x).

Sia data la distribuzione di probabilità la cui PDF è f(x):

-si trovi la sua costante di normalizzazione;
-si trovi la sua media;
-si trovino la varianza e la deviazione standard;
-si trovi la FWHM.
'''

In [ ]:
#Definisco la funzione f, la sua derivata f1 e la funzione integrale generica di una funzione f nell'intervallo [-12,x];
#definisco poi gli array per il campionamento di x, f(x), f'(x).

import numpy as np
import matplotlib.pyplot as plt
eps=10**(-10)
N=10000

def f(x):
    return 8*np.cos(x)**2 * np.exp(-(x-1)**2 /2) + 5/2*np.sin(3/2 *x)**2 * np.exp(-(x+3)**2 /8)

def f1(x):
    return (f(x+eps)-f(x))/eps

def I(f,x):
    A=0
    dt=(x+12)/N
    for k in range(N):
        dA = dt/2 * (f(-12 + k*dt) + f(-12 + (k+1)*dt))
        A+=dA
    return A

X=np.linspace(-12,7,N)
Y=f(X)
Y1=f1(X)

In [ ]:
#Determino il punto di massimo assoluto di f in [-12,7]

xM = round(X[Y.argmax()], 6)
yM = round(max(Y), 6)
M = (xM, yM)

In [ ]:
#Definisco la funzione Z(f,a,b) che come return ha la radice, se esiste, di f in (a,b) (più eventualmente a). Se f non ha
#zeri in [a,b) il return è la lista vuota [], altimenti è [x0] (o, al limite, [a,x0]). Gli zeri di f1 in [-12,7] sono
#ottenuti concatenando l'applicazione di Z - che non funziona se in (a,b) c'è più di uno zero - a n intervallini di misura
#dx = (7+12)/n e in cui si suddivide [-12,7] (n va quindi scelto sufficientemente grande da essere garantiti sul fatto che
#in ogni intervallino ci sia al più uno zero, ma non troppo da inficiare l'efficienza dell'algoritmo).
#Per qualche motivo l'algoritmo descritto fornisce alcune radici replicate, quindi segue un algoritmo che elimina i
#doppioni dalla lista di radici.

def Z(f,a,b):
    z=[]
    a0,b0 = a,b
    
    if f(a)==0:
        z.append(round(a,6))
        a+=eps
    if f(b)==0:
        b-=eps
        
    while abs(b-a)>=eps:
        c=(a+b)/2
        if f(c)==0:
            z.append(round(c,6))
            break
        elif f(a)*f(c)<0:
            b=c
        else:
            a=c

    if abs(b0-c)>=eps:
        z.append(round(c,6))
        
    return z

K,H=[],[]
n=50
dx=19/n
for i in range(n):
    K+= Z(f1,-12+i*dx, -12+(i+1)*dx)
if f1(7)==0:
    K.append(7)

for i in range(len(K)):
    for j in range(i+1,len(K)):
        if K[i]==K[j]:
            H.append(K[i])
for i in range(len(H)):
    K.remove(H[i])

In [ ]:
#Determino, usando la funzione I definita inizialmente, l'integrale di f su [-12,7] e la funzione integrale per
#campionamento.

S = I(f,7)
Y2 = I(f,X)

In [ ]:
#Produco il file di testo richiesto.

file=open('tab.txt','w')
file.write("X      -      f(X)      -      f'(X)      -      F(X)\n\n")
sp = '     '
for k in range(N):
    line = str(round(X[k],8)) +sp+ str(round(Y[k],8)) +sp+ str(round(Y1[k],8)) +sp+ str(round(Y2[k],8))+'\n'
    file.write(line)
file.close()

In [ ]:
#Determino la costante di normalizzazione, la media, la varianza e la deviazione standard usando le informazioni e le
#funzioni già ottenute/definite.

norm = 1/S

def fm(x):
    return x*f(x)
m = I(fm,7)*norm

def fs(x):
    return (x-m)**2*f(x)
s2 = I(fs,7)*norm
s = np.sqrt(s2)

In [ ]:
#Per il calcolo della FWHM uso due metodi diversi (per poi confrontare i risultati): è noto che
#FWHM = x1-x0 ove f(x1)=f(x2)=Ymax/2. Voglio quindi determinare x1 e x2. Con il primo applico l'algoritmo di bisezione
#per ricercare gli zeri di h(x)=f(x)-Ymax/2 rispettivamente nell'intervallo [Xmax-1,Xmax] e [Xmax,Xmax+1] - confidando nel
#fatto che le due radici di h non distino da Xmax più di 1, cosa evidente dal grafico di f. Sottraggo poi i risultati;
#con il secondo confronto, al variare di X[i], i valori di f(X[i]) e f(X[i+1]). Se fra questi due valori si 'sorpassa'
# - ad aumentare o a diminuire - la soglia Ymax/2, si assume x0 (e x1) come la media di X[i] e X[i+1]. L'accuratezza di
#entrambi i metodi si basa sulla bontà della stima di Ymax (quindi dal campionamento); il secondo metodo è afflitta da
#questa dipendenza anche circa la distanza fra X[i] e X[i+1]. Invece il primo, in virtù della bontà dell'algoritmo di
#bisezione, è maggiormente accurato dal punto di vista della ricerca degli zeri di h.

def h(x):
    return f(x)-yM/2

x0, x1 = Z(h,xM-1,xM)[0], Z(h,xM,xM+1)[0]
FWHM=x1-x0

delta=[]
for i in range(N-1):
    if (f(X[i])<yM/2 and f(X[i+1])>yM/2) or (f(X[i])>yM/2 and f(X[i+1])<yM/2):
        delta.append( (X[i]+X[i+1])/2 )
FWHM1 = round(delta[1] - delta[0],6)

In [ ]:
plt.plot(X,Y,'red')
plt.plot([-12,7],[0,0],':',color='black')
plt.title("f(x)",color='red',size=15)
plt.show()

plt.plot(X,Y1,'green')
plt.plot([-12,7],[0,0],':',color='black')
plt.title("f'(x)",color='green',size=15)
plt.show()

plt.plot(X,Y2,'blue')
plt.plot([-12,7],[0,0],':',color='black')
plt.title("F(x)",color='blue',size=15)
plt.show()


print('Il punto di massimo assoluto di f è: M =',M,';')
print("Gli zeri individuati di f' sono: {x_i} =\n",tuple(K))
print('Segue una verifica grafica del risultato ottenuto:')
plt.plot(X,Y1,'green')
plt.plot([-12,7],[0,0],':',color='black')
plt.plot(np.array(K),f1(np.array(K)),'o',color='red')
plt.show()
print("L'integrale esteso a [-12,7] di f è: I =",round(S,6),';\n')

print("Segue che la costante di normalizzazione è: N =",round(norm,6),';')
print('La media della PDF f(x) è: m =',round(m,6),';')
print('La varianza della PDF f(x) è: s2 =',round(s2,6),';')
print('La dev.st. della PDF f(x) è: s =',round(s,6),';')
print('La larghezza a metà altezza della PDF f(x) è (metodo di bisezione): FWHM =',FWHM,';')
print('La larghezza a metà altezza della PDF f(x) è (metodo del cambio di segno): FWHM =',FWHM1,';')
print('Errore: E =', round(100*abs(FWHM-FWHM1)/(min(FWHM,FWHM1)),4),'%;\n')
print('Segue una verifica grafica del risultato ottenuto (col metodo di bisezione):')
plt.plot(X,Y,'red')
plt.plot([x0,x1],[f(x0),f(x1)],'o:b')
plt.title("f(x)",color='red',size=15)
plt.show()

print("\nIl file di testo con i valori tabulati di x, f(x), f'(x) e F(x) è stato generato come tab.txt.\n")
file = open('tab.txt','r')
print(file.read())
file.close()